## Union

In [0]:
data=[(10 ,'Anil',50000, 18),
(11 ,'Vikas',75000,  16),
(12 ,'Nisha',40000,  18),
(13 ,'Nidhi',60000,  17),
(14 ,'Priya',80000,  18),
(15 ,'Mohit',45000,  18),
(16 ,'Rajesh',90000, 10),
(17 ,'Raman',55000, 16),
(18 ,'Sam',65000,   17)]

In [0]:
schema = ['id', 'Name', 'sal', 'mngr_id']
manager_df = spark.createDataFrame(data = data, schema=schema)
manager_df.show()

+---+------+-----+-------+
| id|  Name|  sal|mngr_id|
+---+------+-----+-------+
| 10|  Anil|50000|     18|
| 11| Vikas|75000|     16|
| 12| Nisha|40000|     18|
| 13| Nidhi|60000|     17|
| 14| Priya|80000|     18|
| 15| Mohit|45000|     18|
| 16|Rajesh|90000|     10|
| 17| Raman|55000|     16|
| 18|   Sam|65000|     17|
+---+------+-----+-------+



In [0]:
manager_df.count()

Out[6]: 9

In [0]:
data1=[(19 ,'Sohan',50000, 18),
(20 ,'Sima',75000,  17)]
schema1 = ['id', 'Name', 'sal', 'mngr_id']
manager_df1 = spark.createDataFrame(data = data1, schema=schema1)

In [0]:
manager_df.union(manager_df1).show()

+---+------+-----+-------+
| id|  Name|  sal|mngr_id|
+---+------+-----+-------+
| 10|  Anil|50000|     18|
| 11| Vikas|75000|     16|
| 12| Nisha|40000|     18|
| 13| Nidhi|60000|     17|
| 14| Priya|80000|     18|
| 15| Mohit|45000|     18|
| 16|Rajesh|90000|     10|
| 17| Raman|55000|     16|
| 18|   Sam|65000|     17|
| 19| Sohan|50000|     18|
| 20|  Sima|75000|     17|
+---+------+-----+-------+



In [0]:
manager_df.union(manager_df1).count()

Out[7]: 11

## Union all

In [0]:
dup_data=[(10 ,'Anil',50000, 18),
(11 ,'Vikas',75000,  16),
(12 ,'Nisha',40000,  18),
(13 ,'Nidhi',60000,  17),
(14 ,'Priya',80000,  18),
(15 ,'Mohit',45000,  18),
(16 ,'Rajesh',90000, 10),
(17 ,'Raman',55000, 16),
(18 ,'Sam',65000,   17),
(18 ,'Sam',55000,   17),
(18 ,'Sam',65000,   17)
]

In [0]:
dup_schema = ['id', 'Name', 'sal', 'mngr_id']
duplicate_manager_df = spark.createDataFrame(data = dup_data, schema=dup_schema)
duplicate_manager_df.show()

+---+------+-----+-------+
| id|  Name|  sal|mngr_id|
+---+------+-----+-------+
| 10|  Anil|50000|     18|
| 11| Vikas|75000|     16|
| 12| Nisha|40000|     18|
| 13| Nidhi|60000|     17|
| 14| Priya|80000|     18|
| 15| Mohit|45000|     18|
| 16|Rajesh|90000|     10|
| 17| Raman|55000|     16|
| 18|   Sam|65000|     17|
| 18|   Sam|55000|     17|
| 18|   Sam|65000|     17|
+---+------+-----+-------+



In [0]:
duplicate_manager_df.union(manager_df1).count()

Out[12]: 13

In [0]:
duplicate_manager_df.unionAll(manager_df1).count()

Out[13]: 13

In [0]:
manager_df1.createOrReplaceTempView("manager_df1_tbl")
duplicate_manager_df.createOrReplaceTempView("duplicate_manager_df_tbl")

In [0]:
spark.sql("""
          select * from manager_df1_tbl
          union
          select * from duplicate_manager_df_tbl
          """).count()

Out[18]: 12

In [0]:
spark.sql("""
          select * from manager_df1_tbl
          union all
          select * from duplicate_manager_df_tbl
          """).count()

Out[19]: 13

## Wrong Column Data

In [0]:
wrong_column_data=[(19 ,50000, 18,'Sohan'),
(20 ,75000,  17,'Sima')]
wrong_schema = ['id', 'sal', 'mngr_id', 'Name']
wrong_manager_df = spark.createDataFrame(data = wrong_column_data, schema=wrong_schema)

In [0]:
manager_df1.union(wrong_manager_df).show()

+---+-----+-----+-------+
| id| Name|  sal|mngr_id|
+---+-----+-----+-------+
| 19|Sohan|50000|     18|
| 20| Sima|75000|     17|
| 19|50000|   18|  Sohan|
| 20|75000|   17|   Sima|
+---+-----+-----+-------+



### unionByName

In [0]:
manager_df1.unionByName(wrong_manager_df).show()


+---+-----+-----+-------+
| id| Name|  sal|mngr_id|
+---+-----+-----+-------+
| 19|Sohan|50000|     18|
| 20| Sima|75000|     17|
| 19|Sohan|50000|     18|
| 20| Sima|75000|     17|
+---+-----+-----+-------+



###

In [0]:
wrong_extra_column_data=[(19 ,50000, 18,'Sohan',10),
(20 ,75000,  17,'Sima',20)]
wrong_extra_column_schema = ['id', 'sal', 'mngr_id', 'Name', 'bonus']

wrong_extra_column_df = spark.createDataFrame(data = wrong_extra_column_data, schema=wrong_extra_column_schema)


In [0]:
wrong_extra_column_df.union(manager_df1).show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2253148945278546>:1
----> 1 wrong_extra_column_df.union(manager_df1).show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3646, in DataFrame.union(self, other)
   3598 def union(self, other: "DataFrame") -> "DataFrame":
   3599     """Return a new :class:`DataFrame` containing union of rows in this and another
   3600     :class:`DataFrame`.
   3601 
   (...)
   3644     +----+----+----+
   3645     """
-> 3646     return DataFrame(self._jdf.union

In [0]:
wrong_extra_column_df.select('id', 'sal', 'mngr_id', 'Name').unionByName(manager_df1).show()

+---+-----+-------+-----+
| id|  sal|mngr_id| Name|
+---+-----+-------+-----+
| 19|50000|     18|Sohan|
| 20|75000|     17| Sima|
| 19|50000|     18|Sohan|
| 20|75000|     17| Sima|
+---+-----+-------+-----+



## Wrong Column Name

In [0]:
wrong_column_data2=[(19 ,50000, 18,'Sohan'),
(20 ,75000,  17,'Sima')]
wrong_schema2 = ['id', 'sal', 'mngr_id', 'naam']
wrong_manager_df2 = spark.createDataFrame(data = wrong_column_data2, schema=wrong_schema2)

In [0]:
wrong_manager_df.unionByName(wrong_manager_df2).show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2253148945278550>:1
----> 1 wrong_manager_df.unionByName(wrong_manager_df2).show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3738, in DataFrame.unionByName(self, other, allowMissingColumns)
   3682 def unionByName(self, other: "DataFrame", allowMissingColumns: bool = False) -> "DataFrame":
   3683     """Returns a new :class:`DataFrame` containing union of rows in this and another
   3684     :class:`DataFrame`.
   3685 
   (...)
   3736     +